### Imports

In [2]:
import os
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import sklearn.model_selection
from keras import Sequential
from keras.applications import vgg16
from keras.layers import Dense, Flatten
from keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam

%matplotlib inline




UsageError: Line magic function `%` not found.


### Check for Tensorflow GPU

In [7]:
import tensorflow as tf

print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))
assert tf.config.list_physical_devices('GPU'), "No GPU device detected"

2.6.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### Constants

In [ ]:
# Get train and test data from folder.
TRAIN_DIRECTORY = Path(
    "data/AFD/AFD（Animal Face Dataset)/face images/Saimiri sciureus")

# Image Specifications
IMG_HEIGHT = 224
IMG_WIDTH = 224
NUM_CHANNELS = 3
IMAGE_DIMENSIONS = (IMG_HEIGHT, IMG_WIDTH)

# Cross Fold Validation
TEST_SIZE = 0.1  # 10% reserved for testing
NUMBER_FOLDS = 5  # k-fold CV
NUMBER_REPEATS = 1  # number of times to repeat k-fold cv

# MODEL PARAMETERS
BATCH_SIZE = 32  # This is keras default
NUM_EPOCHS = 30
LEARNING_RATE = 0.001

# Seed - for reproducibility
SEED = 123


### Load Data

In [ ]:
def load_image_dataset(directory, img_width, img_height):
    X = []
    y = []

    for root, _, files in os.walk(directory):
        if not files:
            continue

        root = Path(root)
        dir_name = root.name

        for file in files:
            file_path = root / file
            assert file_path.is_file()

            img = plt.imread(str(file_path))
            if img is not None:
                # Add to the dataset
                res = cv2.resize(img, dsize=(img_width, img_height))
                X.append(res)
                y.append(int(dir_name))
            else:
                # indicate image was skipped
                print("Unable to read file '{}' Skipping".format(
                    str(file_path.absolute())))

    print(len(X))
    print("First image")
    plt.imshow(X[1])

    # Convert to batch along the 0th axis
    X = np.stack(X, axis=0)
    y = np.stack(y, axis=0)

    return X, y


In [ ]:
def investigate_class_distribution(y, number_folds):
    classes, counts = np.unique(y, return_counts=True)

    # TODO - add 1 for false-positives (not a monkey), 1 for previously unseen individuals
    number_classes = len(classes)

    too_few = np.nonzero(np.where(counts < number_folds, classes, 0))
    print(too_few)

    plt.hist(y, bins=classes)
    plt.xticks(np.arange(min(classes), max(classes) + 1, 1))
    plt.ylabel("Instances")
    plt.xlabel("Class")
    plt.title("Class Frequency")
    plt.show()

    return number_classes


In [ ]:
def get_stratified_train_test_split(X, y, test_size, seed):
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
        X, y, shuffle=True, test_size=test_size, random_state=seed, stratify=y)

    print("Train size: ", len(X_train))
    print("Test size: ", len(X_test))

    return X_train, X_test, y_train, y_test


### Model Architecture

In [ ]:
class VGGModel:
    def __init__(self, epochs, learning_rate, number_classes, img_shape):
        self.model = self.make_model(number_classes=number_classes, img_shape=img_shape)
        self.hist = None
        self.epochs = epochs
        self.learning_rate = learning_rate

    @staticmethod
    def make_model(number_classes, img_shape):
        # See here for implementation https://keras.io/api/applications/vgg/#vgg16-function
        vgg_model = vgg16.VGG16(
            include_top=False,
            weights="imagenet",
            input_shape=img_shape,
            pooling='avg',  # Global average pooling applied.
        )
        # Freeze all layers
        vgg_model.trainable = False
        fully_connected_output = Sequential()

        # Add dense layers to perform prediction
        fully_connected_output.add(Flatten())
        fully_connected_output.add(Dense(units=4096, activation="relu"))
        fully_connected_output.add(Dense(units=4096, activation="relu"))
        fully_connected_output.add(
            Dense(units=number_classes, activation="softmax"))

        # Create final model
        return Sequential([vgg_model, fully_connected_output])

    def compile(self):
        optimizer = Adam(learning_rate=self.learning_rate)

        # sparse as Y values are stored as integers (0-42), rather than one-hot encoded (i.e. 3 class, [1,0,0], [0,1,0], [0,0,1])
        loss = sparse_categorical_crossentropy

        self.model.compile(optimizer=optimizer, loss=loss,
                           metrics=['accuracy'])

    # Pre-Process Data for VGG
    @staticmethod
    def pre_process(X_unprocessed):
        # is performing in slices
        return np.apply_along_axis(func1d=lambda x: vgg16.preprocess_input(x), axis=0, arr=X_unprocessed)

    def fit(self, raw_X, raw_y, val_x, val_y):
        preprocessed_X = self.pre_process(raw_X)
        preprocessed_val_x = self.pre_process(val_x)
        self.hist = self.model.fit(
            x=preprocessed_X,
            y=raw_y,
            epochs=self.epochs,
            verbose=True,
            validation_data=(preprocessed_val_x, val_y))

    def evaluate(self, X_eval, y_eval):
        preprocessed_X = self.pre_process(X_eval)
        return self.model.evaluate(preprocessed_X, y_eval)

    def predict(self, X_pred):
        preprocessed_X = self.pre_process(X_pred)
        return self.model.predict(X_pred)

    def plot_results(self):
        self.plot_accuracy()
        self.plot_loss()

    def plot_accuracy(self):
        plt.plot(self.hist.history['accuracy'])
        plt.title("Model Accuracy")
        plt.ylabel("Accuracy")
        plt.xlabel("Epoch")
        plt.show()

    def plot_loss(self):
        plt.plot(self.hist.history['loss'])
        plt.title("Model Loss")
        plt.ylabel("Loss")
        plt.xlabel("Epoch")
        plt.show()


In [ ]:
def train_and_eval_model(wrapped_keras_model: VGGModel, X_train, y_train, X_val, y_val: np.ndarray):
    """
    Trains and Evaluates the input wrapped Keras model.
    :param wrapped_keras_model: The model object should implement the compile, fit, and
    evaluate methods.
    :param X_train: The training dataset to be used, as a 4d np array
    :param y_train: The training dataset labels to be used. Should be sparsely encoded.
    :param X_val: The validation dataset to be used, as a 4d np array
    :param y_val: The validation dataset labels to be used. Should be sparsely encoded.
    :return: The score (typically loss & accuracy) of the model of the fitted model on the validation dataset
    """
    wrapped_keras_model.compile()

    # Could split into batches here.
    print("Fitting Model:")
    wrapped_keras_model.fit(X_train, y_train, X_val, y_val)

    print()
    print("Evaluating Model")
    wrapped_keras_model_score = wrapped_keras_model.evaluate(X_val, y_val)

    print()
    print("Model Score")
    metric_names = wrapped_keras_model.model.metrics_names
    for j in range(len(metric_names)):
        print(
            "- {}: {:.2f}".format(metric_names[j], wrapped_keras_model_score[j]))

    wrapped_keras_model.plot_results()

    return wrapped_keras_model_score


In [ ]:
def repeated_stratified_k_fold(num_repeats, num_folds, seed, X, y, number_classes):
    rskf = sklearn.model_selection.RepeatedStratifiedKFold(
        n_repeats=num_repeats,
        n_splits=num_folds,
        random_state=seed,
    )

    scores = np.zeros(shape=(rskf.get_n_splits(X, y), 2))

    for i, (train_index, test_index) in enumerate(rskf.split(X, y)):
        print("*** Performing Repeat {}/{} Fold {}/{}***".format(round(i / (NUMBER_FOLDS + 1)) + 1,
                                                                 NUMBER_REPEATS,
                                                                 (i + 1) % (NUMBER_FOLDS + 1),
                                                                 NUMBER_FOLDS), )
        X_train, X_val = X[train_index], X[test_index]
        y_train, y_val = y[train_index], y[test_index]

        model = VGGModel(epochs=NUM_EPOCHS, number_classes=number_classes,
                         img_shape=(IMG_HEIGHT, IMG_WIDTH, NUM_CHANNELS))
        score = train_and_eval_model(wrapped_keras_model=model,
                                     X_train=X_train,
                                     y_train=y_train,
                                     X_val=X_val,
                                     y_val=y_val)
        scores[i, 0] = score[0]
        scores[i, 1] = score[1]

        print("\n\n", end="")

    return scores


#### Train Final Model using All Data and Compare to Test Set

In [ ]:
def train_and_evaluate_full_model(X_train, X_test, y_train, y_test, number_classes):
    final_model = VGGModel(epochs=NUM_EPOCHS, number_classes=number_classes,
                           img_shape=(IMG_HEIGHT, IMG_WIDTH, NUM_CHANNELS))
    test_score = train_and_eval_model(wrapped_keras_model=final_model,
                                      X_train=X_train,
                                      y_train=y_train,
                                      X_val=X_test,
                                      y_val=y_test)

    return final_model, test_score


In [ ]:
X, y = load_image_dataset(directory=TRAIN_DIRECTORY,
                          img_width=IMG_WIDTH, img_height=IMG_HEIGHT)


In [ ]:
number_classes = investigate_class_distribution(y=y, number_folds=NUMBER_FOLDS)


In [ ]:
X_train, X_test, y_train, y_test = get_stratified_train_test_split(
    X=X, y=y, test_size=TEST_SIZE, seed=SEED)


In [ ]:
rskf_scores = repeated_stratified_k_fold(X=X_train, y=y_train,
                                         number_classes=number_classes,
                                         num_repeats=NUMBER_REPEATS,
                                         num_folds=NUMBER_FOLDS,
                                         seed=SEED)


In [ ]:
cross_validated_score = np.average(rskf_scores, axis=0)

print("Average Loss: {:.2f}".format(cross_validated_score[0]))
print("Average Accuracy: {:.2f}%".format(cross_validated_score[1] * 100))


In [ ]:
print("Testing Model on Full Dataset:")
final_model, final_model_score = train_and_evaluate_full_model(X_train=X_train,
                                                               y_train=y_train,
                                                               X_test=X_test,
                                                               y_test=y_test,
                                                               number_classes=number_classes)

print("Full Training Dataset Average Loss: {:.2f}".format(
    final_model_score[0]))
print("Full Training Dataset Average Accuracy: {:.2f}%".format(
    final_model_score[1] * 100))
